# [NAPARI](https://napari.org/) testing
## and adding [cellpose](https://cellpose-napari.readthedocs.io/en/latest/index.html) (installer [here](https://pypi.org/project/cellpose-napari/0.1.3/))

To upgrade run in a cell:
```python
!pip install cellpose-napari==0.1.3
!pip install cellpose --upgrade
```

```bash
napari -w cellpose-napari
```

In [ ]:
## to install
# !pip install napari[all]
# !pip install QT
# !pip install napari[pyqt5]
# !pip install cellpose-napari==0.1.3
# !pip install napari-nikon-nd2
# !pip install magicgui
# !pip install napari-pyclesperanto-assistant

## to upgrade:
# !pip install cellpose-napari==0.1.3
# !pip install cellpose --upgrade
# !pip install napari[all] --upgrade

### Basic usage:
```python
viewer = napari.view_image(data.astronaut(), rgb=True)
nbscreenshot(viewer)
viewer.close()
```

example notebooks [here](https://github.com/sofroniewn/napari-training-course/blob/master/lessons/)

In [ ]:
import napari
import cellpose_napari
from napari.utils import nbscreenshot
from tifffile import imread
import numpy as np
from scipy import ndimage
from scipy.stats import mannwhitneyu
import napari_nikon_nd2
import os 
from magicgui import magicgui
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.font_manager as font_manager
import importlib
import svgutils
from svgutils.compose import *
from cmcrameri import cm #colormaps

In [ ]:
plotStyle = 'dark';
# plotStyle = 'light';


batlow = cm.vik(range(255))
batlow = batlow[0:255:np.int(np.floor(255/8)),:]
matplotlib.rcParams['axes.prop_cycle'] = matplotlib.cycler(color=batlow) 

#gene Colors
zfC = {
    'rho'  : '#7d7d7d',
    'sws1' : '#B73AB9',
    'sws2' : '#4364F6',
    'mws1' : '#59CB3B',
    'mws2' : '#59CB3B',
    'mws3' : '#59CB3B',
    'mws4' : '#59CB3B',
    'lws1' : '#CE2A22',
    'lws2' : '#CE2A22',
    'actb2': '#926645',
    'tbx2a': '#c92675',
    'tbx2b': '#7526c9',
    'six7' : '#d6ab00',
}

if plotStyle=='dark':
    # dark background
    params = {"ytick.color" : "w",
              "xtick.color" : "w",
              "axes.labelcolor" : "w",
              "axes.edgecolor" : "w",
             "axes.linewidth" : 3,
             "xtick.major.width" : 3,
             "ytick.major.width" : 3,
             "xtick.major.size" : 8,
             "ytick.major.size" : 8,
             "text.color" : "w"}
    plt.rcParams.update(params)
elif plotStyle=='light':
    # white background
    params = {"ytick.color" : "k",
              "xtick.color" : "k",
              "axes.labelcolor" : "k",
              "axes.edgecolor" : "k",
             "axes.linewidth" : 3,
             "xtick.major.width" : 3,
             "ytick.major.width" : 3,
             "xtick.major.size" : 8,
             "ytick.major.size" : 8,
             "text.color" : "k"}
    plt.rcParams.update(params)

def formatFigureMain(figH, axH, plotH):
    font_path = '/System/Library/Fonts/Avenir.ttc'
    fontTicks = font_manager.FontProperties(fname=font_path, size=22)
    fontLabels = font_manager.FontProperties(fname=font_path, size=28)
    fontTitle = font_manager.FontProperties(fname=font_path, size=32)
    axH.set_xscale('linear')
    axH.spines['top'].set_visible(False)
    axH.spines['right'].set_visible(False)
    
    for label in (axH.get_xticklabels() + axH.get_yticklabels()):
        label.set_fontproperties(fontTicks)
    axH.set_xlabel(axH.get_xlabel(), fontproperties = fontTicks)
    axH.set_ylabel(axH.get_ylabel(), fontproperties = fontTicks)
    return fontLabels

# viewer = napari.Viewer()

print('Opened viewer; plot style is ' + plotStyle)

In [ ]:
# plotStyle = 'dark';

In [ ]:
viewer = napari.Viewer()

In [ ]:
viewer.close()

***
# Index <a id='Index'>∮</a>
***
- [Extraction of z planes for analysis](#zExtract)
- [Segmentation with cellpose](#cellSeg)
- [Manual correction of segmentation](#manualCuration)
- [Create thumbnails _WIP_](#thumbnails)
- [Quantification](#quantification)

***
## Extract layers from z-stacks<a id='zExtract'>∮</a>
***
[Back to Index](#Index)

In [ ]:
# clear viewer
viewer.layers.select_all()
viewer.layers.remove_selected()

In [ ]:
# clear viewer
viewer.layers.select_all()
viewer.layers.remove_selected()

# analysis directory
dAnalysis = "/Users/angueyraaristjm/Documents/LiImaging/Analysis/CRtbx2F0s/m2Gs2C/"

# open file


# uninjected
dPath = "/Volumes/zfSSD/LiImaging/A1R/zf/20201113_CRtbx2b/"
### 20201009: imaged 3dpf and 5dpf larvae. Analyzing 5 dpf for paper
fPath = "20201113_5dpf_Rh2hG_s2mCh_uninj_i07b"; zlims = [39,43]; gene = 'wt'; # a little curved and nerve head is included
fPath = "20201113_5dpf_Rh2hG_s2mCh_uninj_i07d"; zlims=[29,32]; gene = "wt";
fPath = "20201113_5dpf_Rh2hG_s2mCh_uninj_i08b"; zlims=[30,32]; gene = "wt";  #EXAMPLE???
#### fPath = "20201113_5dpf_Rh2hG_s2mCh_uninj_i08b2"; zlims=[00,00]; gene = "wt";  # beautiful peripheral retina
fPath = "20201113_5dpf_Rh2hG_s2mCh_uninj_i08d"; zlims=[28,30]; gene = "wt";
fPath = "20201113_5dpf_Rh2hG_s2mCh_uninj_i09b"; zlims=[36,39]; gene = "wt";
fPath = "20201113_5dpf_Rh2hG_s2mCh_uninj_i09d"; zlims=[32,34]; gene = "wt";
#### fPath = "20201113_5dpf_Rh2hG_s2mCh_uninj_i10b"; zlims=[00,00]; gene = "wt"; #side-view
fPath = "20201113_5dpf_Rh2hG_s2mCh_uninj_i10d"; zlims=[28,31]; gene = "wt";
#### fPath = "20201113_5dpf_Rh2hG_s2mCh_uninj_i10e"; zlims=[00,00]; gene = "wt"; #side-view
fPath = "20201113_5dpf_Rh2hG_s2mCh_uninjected_i11b"; zlims=[27,29]; gene = "wt";
fPath = "20201113_5dpf_Rh2hG_s2mCh_uninjected_i14b"; zlims=[26,29]; gene = "wt";

#tbx2b
# dPath = "/Volumes/zfSSD/LiImaging/A1R/zf/20201113_CRtbx2b/"
fPath = "20201113_5dpf_Rh2hG_s2mCh_CRtbx2b_i04b"; zlims=[27,31]; gene = "tbx2b";
fPath = "20201113_5dpf_Rh2hG_s2mCh_CRtbx2b_i04d"; zlims=[29,31]; gene = "tbx2b";
fPath = "20201113_5dpf_Rh2hG_s2mCh_CRtbx2b_i05b"; zlims=[32,34]; gene = "tbx2b";
fPath = "20201113_5dpf_Rh2hG_s2mCh_CRtbx2b_i05d"; zlims=[26,29]; gene = "tbx2b";
fPath = "20201113_5dpf_Rh2hG_s2mCh_CRtbx2b_i06b"; zlims=[22,23]; gene = "tbx2b"; #extreme phenotype example
fPath = "20201113_5dpf_Rh2hG_s2mCh_CRtbx2b_i06d"; zlims=[42,44]; gene = "tbx2b";
fPath = "20201113_5dpf_Rh2hG_s2mCh_CRtbx2b_i07b"; zlims=[25,27]; gene = "tbx2b";  #example for injected but no phenotype?
fPath = "20201113_5dpf_Rh2hG_s2mCh_CRtbx2b_i07d"; zlims=[28,30]; gene = "tbx2b";
fPath = "20201113_5dpf_Rh2hG_s2mCh_CRtbx2b_i08b"; zlims=[40,42]; gene = "tbx2b";
fPath = "20201113_5dpf_Rh2hG_s2mCh_CRtbx2b_i08d"; zlims=[29,31]; gene = "tbx2b";

# uninjected
dPath = "/Volumes/angueyraNEI/LiImaging/A1R/zf2021onwards/20210108_CRtbx2a/"
fPath = "20210108_5dpf_Rh2hG_s2mCh_uninjected_i01b"; zlims=[39,42]; gene = "wt";
fPath = "20210108_5dpf_Rh2hG_s2mCh_uninjected_i02b"; zlims=[27,30]; gene = "wt";
fPath = "20210108_5dpf_Rh2hG_s2mCh_uninjected_i03b"; zlims=[34,36]; gene = "wt";
fPath = "20210108_5dpf_Rh2hG_s2mCh_uninjected_i04b"; zlims=[51,53]; gene = "wt";
fPath = "20210108_5dpf_Rh2hG_s2mCh_zpr3_647_uninjected_i06d"; zlims=[27,29]; gene = "wt";
fPath = "20210108_5dpf_Rh2hG_s2mCh_zpr3_647_uninjected_i07b"; zlims=[27,29]; gene = "wt";

### excluded
### fPath = "20210108_5dpf_Rh2hG_s2mCh_uninjected_i01d"; zlims=[27,31]; gene = "wt"; ### really nice side-view
### fPath = "20210108_5dpf_Rh2hG_s2mCh_uninjected_i01e"; zlims=[27,31]; gene = "wt"; ### another nice side-view
### fPath = "20210108_5dpf_Rh2hG_s2mCh_uninjected_i02d"; zlims=[27,31]; gene = "wt"; ### side-view
### fPath = "20210108_5dpf_Rh2hG_s2mCh_uninjected_i03d"; zlims=[27,31]; gene = "wt"; ### side-view
### fPath = "20210108_5dpf_Rh2hG_s2mCh_uninjected_i04bprime"; zlims=[27,31]; gene = "wt"; # less zoom
### fPath = "20210108_5dpf_Rh2hG_s2mCh_uninjected_i04d"; zlims=[27,31]; gene = "wt"; ### side-view
### fPath = "20210108_5dpf_Rh2hG_s2mCh_zpr3_647_uninjected_i06b"; zlims=[27,31]; gene = "wt"; ### not central retina

# # tbx2a
dPath = "/Volumes/angueyraNEI/LiImaging/A1R/zf2021onwards/20210108_CRtbx2a/"
fPath = "20210108_5dpf_Rh2hG_s2mCh_CRtbx2a_i01b"; zlims=[46,49]; gene = "tbx2a"; # curved retina, will require manual curation on big sections
fPath = "20210108_5dpf_Rh2hG_s2mCh_CRtbx2a_i01d"; zlims=[27,29]; gene = "tbx2a"; # can't open it; had to do it in FIJI
fPath = "20210108_5dpf_Rh2hG_s2mCh_CRtbx2a_i02b"; zlims=[27,29]; gene = "tbx2a"; # can't open it; had to do it in FIJI
fPath = "20210108_5dpf_Rh2hG_s2mCh_CRtbx2a_i02d"; zlims=[0,0]; gene = "tbx2a"; # can't open it; had to do it in FIJI
fPath = "20210108_5dpf_Rh2hG_s2mCh_mRho647_CRtbx2a_i20b"; zlims=[23,25]; gene = "tbx2a"; # can't open it; had to do it in FIJI
fPath = "20210108_5dpf_Rh2hG_s2mCh_mRho647_CRtbx2a_i21b"; zlims=[40,42]; gene = "tbx2a";
fPath = "20210108_5dpf_Rh2hG_s2mCh_mRho647_CRtbx2a_i21d"; zlims=[39,41]; gene = "tbx2a";
fPath = "20210108_5dpf_Rh2hG_s2mCh_mRho647_CRtbx2a_i25b"; zlims=[33,35]; gene = "tbx2a"; # a little curved, nerve head visible
fPath = "20210108_5dpf_Rh2hG_s2mCh_zpr3_647_CRtbx2a_i13b"; zlims=[27,31]; gene = "tbx2a";


# create subdirectory for analysis
dOut = dAnalysis + gene + '/' + fPath + '/'
if (os.path.isdir(dOut)==False):
    os.mkdir(dOut)

print('Canvas clear')

In [ ]:
# load whole stack to figure out best planes for cellpose
viewer.open((dPath+fPath + ".nd2"))
nChannels = len(viewer.layers);
viewer.layers.remove(viewer.layers[nChannels-1]) # remove transmitted detector image
viewer.layers[0].colormap = 'green'
viewer.layers[0].name = 'M'
viewer.layers[1].colormap = 'magenta'
viewer.layers[1].name = 'S'
if (nChannels > 3):
    viewer.layers[2].colormap = 'bop orange'
    viewer.layers[2].name = 'X'
viewer.layers.select_next();
print('Loaded!')

In [ ]:
# make mips (and remove any previous ones)
zlims = [21,23];
if len(viewer.layers)>2:
    for l in viewer.layers[2:]:
        viewer.layers.remove(l)

for l in [viewer.layers[0],viewer.layers[1]]:
    l.visible = False
#     viewer.layers.remove(l.name + '_mip')
    viewer.add_image(l.data[zlims[0]:zlims[1]].max(axis=0), blending='additive', colormap = l.colormap, name = l.name + "_mip")

In [ ]:
# when things look good, save in folder for batch cellpose analysis
l = viewer.layers['M_mip']; l.save(dOut + l.name + '.tiff')
l = viewer.layers['S_mip']; l.save(dOut + l.name + '.tiff')

***
## Segmentation using cellpose<a id='cellSeg'>∮</a>
***
[Back to Index](#Index)

In [ ]:
# clear viewer
viewer.layers.select_all()
viewer.layers.remove_selected()

# analysis directory
dAnalysis = "/Users/angueyraaristjm/Documents/LiImaging/Analysis/CRtbx2F0s/m2Gs2C/"

# open file
# uninjected
dPath = "/Volumes/zfSSD/LiImaging/A1R/zf/20201113_CRtbx2b/"
### 20201009: imaged 3dpf and 5dpf larvae. Analyzing 5 dpf for paper
fPath = "20201113_5dpf_Rh2hG_s2mCh_uninj_i07b"; zlims = [39,43]; gene = 'wt'; # a little curved and nerve head is included
fPath = "20201113_5dpf_Rh2hG_s2mCh_uninj_i07d"; zlims=[29,32]; gene = "wt";
fPath = "20201113_5dpf_Rh2hG_s2mCh_uninj_i08b"; zlims=[30,32]; gene = "wt";  #EXAMPLE???
fPath = "20201113_5dpf_Rh2hG_s2mCh_uninj_i08d"; zlims=[28,30]; gene = "wt";
fPath = "20201113_5dpf_Rh2hG_s2mCh_uninj_i09b"; zlims=[36,39]; gene = "wt";
fPath = "20201113_5dpf_Rh2hG_s2mCh_uninj_i09d"; zlims=[32,34]; gene = "wt";
fPath = "20201113_5dpf_Rh2hG_s2mCh_uninj_i10d"; zlims=[28,31]; gene = "wt";
fPath = "20201113_5dpf_Rh2hG_s2mCh_uninjected_i11b"; zlims=[27,29]; gene = "wt"; # S cone channel is strange in this one
fPath = "20201113_5dpf_Rh2hG_s2mCh_uninjected_i14b"; zlims=[26,29]; gene = "wt";

#### fPath = "20201113_5dpf_Rh2hG_s2mCh_uninj_i08b2"; zlims=[00,00]; gene = "wt";  # beautiful peripheral retina
#### fPath = "20201113_5dpf_Rh2hG_s2mCh_uninj_i10b"; zlims=[00,00]; gene = "wt"; #side-view
#### fPath = "20201113_5dpf_Rh2hG_s2mCh_uninj_i10e"; zlims=[00,00]; gene = "wt"; #side-view


#tbx2b
#  is there a decrease in S cones?
fPath = "20201113_5dpf_Rh2hG_s2mCh_CRtbx2b_i04b"; zlims=[27,31]; gene = "tbx2b";
fPath = "20201113_5dpf_Rh2hG_s2mCh_CRtbx2b_i04d"; zlims=[29,31]; gene = "tbx2b";
fPath = "20201113_5dpf_Rh2hG_s2mCh_CRtbx2b_i05b"; zlims=[32,34]; gene = "tbx2b";
fPath = "20201113_5dpf_Rh2hG_s2mCh_CRtbx2b_i05d"; zlims=[26,29]; gene = "tbx2b";
fPath = "20201113_5dpf_Rh2hG_s2mCh_CRtbx2b_i06b"; zlims=[22,23]; gene = "tbx2b"; #extreme phenotype example
fPath = "20201113_5dpf_Rh2hG_s2mCh_CRtbx2b_i06d"; zlims=[42,44]; gene = "tbx2b";
fPath = "20201113_5dpf_Rh2hG_s2mCh_CRtbx2b_i07b"; zlims=[25,27]; gene = "tbx2b";  #example for injected but no phenotype?
fPath = "20201113_5dpf_Rh2hG_s2mCh_CRtbx2b_i07d"; zlims=[28,30]; gene = "tbx2b";
fPath = "20201113_5dpf_Rh2hG_s2mCh_CRtbx2b_i08b"; zlims=[40,42]; gene = "tbx2b";
fPath = "20201113_5dpf_Rh2hG_s2mCh_CRtbx2b_i08d"; zlims=[29,31]; gene = "tbx2b";

###
# uninjected
dPath = "/Volumes/angueyraNEI/LiImaging/A1R/zf2021onwards/20210108_CRtbx2a/"
fPath = "20210108_5dpf_Rh2hG_s2mCh_uninjected_i01b"; zlims=[39,42]; gene = "wt";
fPath = "20210108_5dpf_Rh2hG_s2mCh_uninjected_i02b"; zlims=[27,30]; gene = "wt";
fPath = "20210108_5dpf_Rh2hG_s2mCh_uninjected_i03b"; zlims=[34,36]; gene = "wt";
fPath = "20210108_5dpf_Rh2hG_s2mCh_uninjected_i04b"; zlims=[51,53]; gene = "wt";
fPath = "20210108_5dpf_Rh2hG_s2mCh_zpr3_647_uninjected_i06d"; zlims=[27,29]; gene = "wt";
fPath = "20210108_5dpf_Rh2hG_s2mCh_zpr3_647_uninjected_i07b"; zlims=[27,29]; gene = "wt";

### excluded
### fPath = "20210108_5dpf_Rh2hG_s2mCh_uninjected_i01d"; zlims=[27,31]; gene = "wt"; ### really nice side-view
### fPath = "20210108_5dpf_Rh2hG_s2mCh_uninjected_i01e"; zlims=[27,31]; gene = "wt"; ### another nice side-view
### fPath = "20210108_5dpf_Rh2hG_s2mCh_uninjected_i02d"; zlims=[27,31]; gene = "wt"; ### side-view
### fPath = "20210108_5dpf_Rh2hG_s2mCh_uninjected_i03d"; zlims=[27,31]; gene = "wt"; ### side-view
### fPath = "20210108_5dpf_Rh2hG_s2mCh_uninjected_i04bprime"; zlims=[27,31]; gene = "wt"; # less zoom
### fPath = "20210108_5dpf_Rh2hG_s2mCh_uninjected_i04d"; zlims=[27,31]; gene = "wt"; ### side-view
### fPath = "20210108_5dpf_Rh2hG_s2mCh_zpr3_647_uninjected_i06b"; zlims=[27,31]; gene = "wt"; ### not central retina

# # tbx2a
# dPath = "/Volumes/angueyraNEI/LiImaging/A1R/zf2021onwards/20210108_CRtbx2a/"
fPath = "20210108_5dpf_Rh2hG_s2mCh_CRtbx2a_i01b"; zlims=[46,49]; gene = "tbx2a"; # curved retina, will require manual curation on big sections
fPath = "20210108_5dpf_Rh2hG_s2mCh_CRtbx2a_i01d"; zlims=[27,29]; gene = "tbx2a"; # can't open it; had to do it in FIJI. There seem to be extra dim S cones here
fPath = "20210108_5dpf_Rh2hG_s2mCh_CRtbx2a_i02b"; zlims=[27,29]; gene = "tbx2a"; # can't open it; had to do it in FIJI
fPath = "20210108_5dpf_Rh2hG_s2mCh_CRtbx2a_i02d"; zlims=[0,0]; gene = "tbx2a"; # can't open it; had to do it in FIJI
fPath = "20210108_5dpf_Rh2hG_s2mCh_mRho647_CRtbx2a_i20b"; zlims=[23,25]; gene = "tbx2a"; # can't open it; had to do it in FIJI
fPath = "20210108_5dpf_Rh2hG_s2mCh_mRho647_CRtbx2a_i21b"; zlims=[40,42]; gene = "tbx2a"; # will require a lot of curation
fPath = "20210108_5dpf_Rh2hG_s2mCh_mRho647_CRtbx2a_i21d"; zlims=[39,41]; gene = "tbx2a";
fPath = "20210108_5dpf_Rh2hG_s2mCh_mRho647_CRtbx2a_i25b"; zlims=[33,35]; gene = "tbx2a"; # a little curved, nerve head visible
fPath = "20210108_5dpf_Rh2hG_s2mCh_zpr3_647_CRtbx2a_i13b"; zlims=[27,31]; gene = "tbx2a";

# create subdirectory for analysis
dOut = dAnalysis + gene + '/' + fPath + '/'

print('Canvas clear')

In [ ]:
viewer.layers.select_all()
viewer.layers.remove_selected()

# load mips
viewer.open(dOut + 'M' + "_mip.tiff", plugin='builtins', colormap = 'green');
viewer.open(dOut + 'S' + "_mip.tiff", plugin='builtins', colormap = 'magenta', blending='additive');
print('Loaded! Now run cellpose!!!!')

In [ ]:
# after running cellpose, rename cellpose images and save before manual curation:
baseName = 'M'
lname = baseName + '_mip'
viewer.layers[lname + '_cp_masks_000'].name = baseName + '_seg'
viewer.layers[lname + '_cp_outlines_000'].name = baseName + '_outlines'
viewer.layers[lname + '_cp_cellprob_000'].name = baseName + '_prob'
viewer.layers[lname + '_cp_flows_000'].name = baseName + '_flows'

l = viewer.layers[baseName + '_seg']; l.save(dOut + l.name + '.tiff')
l = viewer.layers[baseName + '_outlines']; l.save(dOut + l.name + '.tiff')
l = viewer.layers[baseName + '_prob']; l.save(dOut + l.name + '.tiff')
l = viewer.layers[baseName + '_flows']; l.save(dOut + l.name + '.tiff')

In [ ]:
# after running cellpose, rename cellpose images and save before manual curation:
baseName = 'S'
lname = baseName + '_mip'
viewer.layers[lname + '_cp_masks_000'].name = baseName + '_seg'
viewer.layers[lname + '_cp_outlines_000'].name = baseName + '_outlines'
viewer.layers[lname + '_cp_cellprob_000'].name = baseName + '_prob'
viewer.layers[lname + '_cp_flows_000'].name = baseName + '_flows'

l = viewer.layers[baseName + '_seg']; l.save(dOut + l.name + '.tiff')
l = viewer.layers[baseName + '_outlines']; l.save(dOut + l.name + '.tiff')
l = viewer.layers[baseName + '_prob']; l.save(dOut + l.name + '.tiff')
l = viewer.layers[baseName + '_flows']; l.save(dOut + l.name + '.tiff')

***
## Manual correction of cellpose segmentation<a id='manualCuration'>∮</a>
***
[Back to Index](#Index)

In [ ]:
# analysis directory
dAnalysis = "/Users/angueyraaristjm/Documents/LiImaging/Analysis/CRtbx2F0s/m2Gs2C/"

# open file
# uninjected
dPath = "/Volumes/zfSSD/LiImaging/A1R/zf/20201113_CRtbx2b/"
### 20201009: imaged 3dpf and 5dpf larvae. Analyzing 5 dpf for paper
fPath = "20201113_5dpf_Rh2hG_s2mCh_uninj_i07b"; zlims = [39,43]; gene = 'wt'; #done with M complete # a little curved and nerve head is included
fPath = "20201113_5dpf_Rh2hG_s2mCh_uninj_i07d"; zlims=[29,32]; gene = "wt"; #done with M complete
fPath = "20201113_5dpf_Rh2hG_s2mCh_uninj_i08b"; zlims=[30,32]; gene = "wt";  #done with M complete #EXAMPLE???
fPath = "20201113_5dpf_Rh2hG_s2mCh_uninj_i08d"; zlims=[28,30]; gene = "wt"; #done with M complete
fPath = "20201113_5dpf_Rh2hG_s2mCh_uninj_i09b"; zlims=[36,39]; gene = "wt"; #done with M complete
fPath = "20201113_5dpf_Rh2hG_s2mCh_uninj_i09d"; zlims=[32,34]; gene = "wt"; #done with M complete
fPath = "20201113_5dpf_Rh2hG_s2mCh_uninj_i10d"; zlims=[28,31]; gene = "wt"; #done S only
fPath = "20201113_5dpf_Rh2hG_s2mCh_uninjected_i11b"; zlims=[27,29]; gene = "wt"; #done S only # S cone channel is strange in this one
fPath = "20201113_5dpf_Rh2hG_s2mCh_uninjected_i14b"; zlims=[26,29]; gene = "wt"; #done S only

#### fPath = "20201113_5dpf_Rh2hG_s2mCh_uninj_i08b2"; zlims=[00,00]; gene = "wt";  # beautiful peripheral retina
#### fPath = "20201113_5dpf_Rh2hG_s2mCh_uninj_i10b"; zlims=[00,00]; gene = "wt"; #side-view
#### fPath = "20201113_5dpf_Rh2hG_s2mCh_uninj_i10e"; zlims=[00,00]; gene = "wt"; #side-view

# tbx2a


#tbx2b
#  is there a decrease in S cones?
fPath = "20201113_5dpf_Rh2hG_s2mCh_CRtbx2b_i04b"; zlims=[27,31]; gene = "tbx2b"; #done with M complete
fPath = "20201113_5dpf_Rh2hG_s2mCh_CRtbx2b_i04d"; zlims=[29,31]; gene = "tbx2b"; # done with S
fPath = "20201113_5dpf_Rh2hG_s2mCh_CRtbx2b_i05b"; zlims=[32,34]; gene = "tbx2b"; # done with S
fPath = "20201113_5dpf_Rh2hG_s2mCh_CRtbx2b_i05d"; zlims=[26,29]; gene = "tbx2b"; # done with S
fPath = "20201113_5dpf_Rh2hG_s2mCh_CRtbx2b_i06b"; zlims=[22,23]; gene = "tbx2b"; # done with S #extreme phenotype example
fPath = "20201113_5dpf_Rh2hG_s2mCh_CRtbx2b_i06d"; zlims=[42,44]; gene = "tbx2b"; # done with S
fPath = "20201113_5dpf_Rh2hG_s2mCh_CRtbx2b_i07b"; zlims=[25,27]; gene = "tbx2b"; # done with S #example for injected but no phenotype?
fPath = "20201113_5dpf_Rh2hG_s2mCh_CRtbx2b_i07d"; zlims=[28,30]; gene = "tbx2b"; # done with S
fPath = "20201113_5dpf_Rh2hG_s2mCh_CRtbx2b_i08b"; zlims=[40,42]; gene = "tbx2b"; # done with S
fPath = "20201113_5dpf_Rh2hG_s2mCh_CRtbx2b_i08d"; zlims=[29,31]; gene = "tbx2b"; # done with S

### these are not done (Jul 2021)
# uninjected
dPath = "/Volumes/angueyraNEI/LiImaging/A1R/zf2021onwards/20210108_CRtbx2a/"
fPath = "20210108_5dpf_Rh2hG_s2mCh_uninjected_i01b"; zlims=[39,42]; gene = "wt";
fPath = "20210108_5dpf_Rh2hG_s2mCh_uninjected_i02b"; zlims=[27,30]; gene = "wt";
fPath = "20210108_5dpf_Rh2hG_s2mCh_uninjected_i03b"; zlims=[34,36]; gene = "wt";
fPath = "20210108_5dpf_Rh2hG_s2mCh_uninjected_i04b"; zlims=[51,53]; gene = "wt";
fPath = "20210108_5dpf_Rh2hG_s2mCh_zpr3_647_uninjected_i06d"; zlims=[27,29]; gene = "wt";
fPath = "20210108_5dpf_Rh2hG_s2mCh_zpr3_647_uninjected_i07b"; zlims=[27,29]; gene = "wt";

### excluded
### fPath = "20210108_5dpf_Rh2hG_s2mCh_uninjected_i01d"; zlims=[27,31]; gene = "wt"; ### really nice side-view
### fPath = "20210108_5dpf_Rh2hG_s2mCh_uninjected_i01e"; zlims=[27,31]; gene = "wt"; ### another nice side-view
### fPath = "20210108_5dpf_Rh2hG_s2mCh_uninjected_i02d"; zlims=[27,31]; gene = "wt"; ### side-view
### fPath = "20210108_5dpf_Rh2hG_s2mCh_uninjected_i03d"; zlims=[27,31]; gene = "wt"; ### side-view
### fPath = "20210108_5dpf_Rh2hG_s2mCh_uninjected_i04bprime"; zlims=[27,31]; gene = "wt"; # less zoom
### fPath = "20210108_5dpf_Rh2hG_s2mCh_uninjected_i04d"; zlims=[27,31]; gene = "wt"; ### side-view
### fPath = "20210108_5dpf_Rh2hG_s2mCh_zpr3_647_uninjected_i06b"; zlims=[27,31]; gene = "wt"; ### not central retina

# # tbx2a
dPath = "/Volumes/angueyraNEI/LiImaging/A1R/zf2021onwards/20210108_CRtbx2a/"
fPath = "20210108_5dpf_Rh2hG_s2mCh_CRtbx2a_i01b"; zlims=[46,49]; gene = "tbx2a"; # curved retina, will require manual curation on big sections
fPath = "20210108_5dpf_Rh2hG_s2mCh_CRtbx2a_i01d"; zlims=[27,29]; gene = "tbx2a"; # can't open it; had to do it in FIJI. There seem to be extra dim S cones here
fPath = "20210108_5dpf_Rh2hG_s2mCh_CRtbx2a_i02b"; zlims=[27,29]; gene = "tbx2a"; # can't open it; had to do it in FIJI
fPath = "20210108_5dpf_Rh2hG_s2mCh_CRtbx2a_i02d"; zlims=[0,0]; gene = "tbx2a"; # can't open it; had to do it in FIJI
fPath = "20210108_5dpf_Rh2hG_s2mCh_mRho647_CRtbx2a_i20b"; zlims=[23,25]; gene = "tbx2a"; # can't open it; had to do it in FIJI
fPath = "20210108_5dpf_Rh2hG_s2mCh_mRho647_CRtbx2a_i21b"; zlims=[40,42]; gene = "tbx2a"; # will require a lot of curation
fPath = "20210108_5dpf_Rh2hG_s2mCh_mRho647_CRtbx2a_i21d"; zlims=[39,41]; gene = "tbx2a"; # this could be example to show a lot of GFP but not in S cones
fPath = "20210108_5dpf_Rh2hG_s2mCh_mRho647_CRtbx2a_i25b"; zlims=[33,35]; gene = "tbx2a"; # a little curved, nerve head visible
fPath = "20210108_5dpf_Rh2hG_s2mCh_zpr3_647_CRtbx2a_i13b"; zlims=[27,31]; gene = "tbx2a";


# subdirectory for analysis
dOut = dAnalysis + gene + '/' + fPath + '/'

# clear viewer
viewer.layers.select_all()
viewer.layers.remove_selected()
# clear key binds
@viewer.bind_key('k', overwrite=True)
def toggle_sel(viewer):
    ...
@viewer.bind_key('Shift-x', overwrite=True)
def removeLabel(viewer):
    ...

# load mips
viewer.open(dOut + 'M' + "_mip.tiff", plugin='builtins', colormap = 'green');
viewer.open(dOut + 'S' + "_mip.tiff", plugin='builtins', colormap = 'magenta', blending='additive');
viewer.open(dOut + 'M' + "_seg.tiff", plugin='builtins');
viewer.open(dOut + 'S' + "_seg.tiff", plugin='builtins');
viewer.layers['M_seg'].preserve_labels = True;
viewer.layers['S_seg'].preserve_labels = True;

# viewer.layers['M_seg'].contour = 5
# viewer.layers['S_seg'].contour = 5

viewer.layers['M_mip'].visible = False
viewer.layers['M_seg'].visible = False


print('Loaded  ' + fPath + ' !\n Now fix it and save it!')

In [ ]:
#define useful keyboard shortcuts
@viewer.bind_key('Shift-x', overwrite=True)
def removeLabel(viewer):
    lname = 'S_seg'
    tempd = viewer.layers[lname].data
    tempd[tempd == viewer.layers[lname].selected_label]=0
    viewer.layers[lname].data = tempd
    print('Cut!')

print('removeLabel on S_seg ("Shift-X")')

@viewer.bind_key('k', overwrite=True)
def toggle_sel(viewer):
    lname = 'S_seg'
    if (viewer.layers[lname].preserve_labels == True):
        viewer.layers[lname].preserve_labels = False
    elif (viewer.layers[lname].preserve_labels == False):
        viewer.layers[lname].preserve_labels = True
print('toggle S_seg preserve_labels ("k")')
        
@viewer.bind_key('b', overwrite=True)
def toggle_sel(viewer):
    lname = 'M_mip'
    if (viewer.layers[lname].visible == True):
        viewer.layers[lname].visible = False
    elif (viewer.layers[lname].visible == False):
        viewer.layers[lname].visible = True

print('toggle M_mip visibility ("B")')

In [ ]:
# load z-stack if needed
# dPath = "/Volumes/zfSSD/LiImaging/A1R/zf/20201030_CRtbx2ab/"
# viewer.open((dPath+fPath + ".nd2"))
# viewer.layers.remove(viewer.layers[2]) # remove transmitted detector image
# viewer.layers[fPath].colormap = 'green'
# viewer.layers[fPath].name = 'Mz'
# viewer.layers[fPath+' [1]'].colormap = 'magenta'
# viewer.layers[fPath+' [1]'].name = 'Lz'
# # viewer.layers.select_next();
# print('Loaded!')

In [ ]:
# resave curated segmentation after napari-ing around
baseName = 'S'
lname = baseName + '_seg'
l = viewer.layers[baseName + '_seg']; l.save(dOut + l.name + '_curated.tiff')
viewer.layers['M_mip'].visible = True
viewer.layers['M_seg'].visible = True
viewer.layers['S_seg'].contour = 0
viewer.layers['S_mip'].visible = False
viewer.layers['S_seg'].visible = False
print('Done with ' + lname)

@viewer.bind_key('Shift-x', overwrite=True)
def removeLabel(viewer):
    lname = 'M_seg'
    tempd = viewer.layers[lname].data
    tempd[tempd == viewer.layers[lname].selected_label]=0
    viewer.layers[lname].data = tempd
    print('Cut!')

print('removeLabel on M_seg ("Shift-X")')

@viewer.bind_key('k', overwrite=True)
def toggle_sel(viewer):
    lname = 'M_seg'
    if (viewer.layers[lname].preserve_labels == True):
        viewer.layers[lname].preserve_labels = False
    elif (viewer.layers[lname].preserve_labels == False):
        viewer.layers[lname].preserve_labels = True
print('toggle M_seg preserve_labels ("k")')

@viewer.bind_key('b', overwrite=True)
def toggle_sel(viewer):
    lname = 'S_mip'
    if (viewer.layers[lname].visible == True):
        viewer.layers[lname].visible = False
    elif (viewer.layers[lname].visible == False):
        viewer.layers[lname].visible = True

print('toggle S_mip visibility ("B")')

In [ ]:
# resave curated segmentation after napari-ing around
baseName = 'M'
lname = baseName + '_seg'
l = viewer.layers[baseName + '_seg']; l.save(dOut + l.name + '_curated_incomplete.tiff')
# l = viewer.layers[baseName + '_seg']; l.save(dOut + l.name + '_curated.tiff')
print('Done with ' + lname)

In [ ]:
# remove all except mips
if len(viewer.layers)>2:
    for l in viewer.layers[2:]:
        viewer.layers.remove(l)
viewer.reset_view()

In [ ]:
# viewer.open(dOut + 'M' + "_seg_curated.tiff", name='M_seg', plugin='builtins');
viewer.open(dOut + 'M' + "_seg_curated_incomplete.tiff", name='M_seg', plugin='builtins');
viewer.open(dOut + 'S' + "_seg_curated.tiff", name='S_seg', plugin='builtins');
viewer.layers['S_seg'].contour = 5

print(fPath)
nR = len(np.unique(viewer.layers['M_seg'].data))-1
print("Image has {0} M Cones".format(nR))
nU = len(np.unique(viewer.layers['S_seg'].data))-1
print("Image has {0} S cones".format(nU))
print("{0},{1},{2},{3}".format(fPath,gene,nR,nU))

In [ ]:
@viewer.bind_key('Shift-x', overwrite=True)
def removeLabel(viewer):
    lname = 'M_seg'
#     lname = 'S_seg'
    tempd = viewer.layers[lname].data
    tempd[tempd == viewer.layers[lname].selected_label]=0
    viewer.layers[lname].data = tempd
    print('Cut!')

In [ ]:
# manual merging of labels
mergeLabels = [15,17]
lname = 'M_seg'
# lname = 'S_seg'
tempd = viewer.layers[lname].data
tempd[tempd == mergeLabels[1]]=mergeLabels[0]
viewer.layers[lname].data = tempd
print('Merged!')

In [ ]:
# manually delete label
selLabel = 49
lname = 'S_seg'
tempd = viewer.layers[lname].data
tempd[tempd == selLabel]=0
viewer.layers[lname].data = tempd

In [ ]:
@viewer.bind_key('k', overwrite=True)
def toggle_sel(viewer):
    lname = 'M_seg'
    if (viewer.layers[lname].show_selected_label == True):
        viewer.layers[lname].show_selected_label = False
    elif (viewer.layers[lname].show_selected_label == False):
        viewer.layers[lname].show_selected_label = True

## Would be nice here to export a png with mips and contours of curated segmentation
> and set layers['X_seg'].contour = 3;

In [ ]:
# analysis directory
dAnalysis = "/Users/angueyraaristjm/Documents/LiImaging/Analysis/CRtbx2F0s/m2Gs2C/"

# open file
# uninjected
dPath = "/Volumes/zfSSD/LiImaging/A1R/zf/20201113_CRtbx2b/"
### 20201009: imaged 3dpf and 5dpf larvae. Analyzing 5 dpf for paper
fPath = "20201113_5dpf_Rh2hG_s2mCh_uninj_i07b"; zlims = [39,43]; gene = 'wt'; #done with M complete # a little curved and nerve head is included
fPath = "20201113_5dpf_Rh2hG_s2mCh_uninj_i07d"; zlims=[29,32]; gene = "wt"; #done with M complete
fPath = "20201113_5dpf_Rh2hG_s2mCh_uninj_i08b"; zlims=[30,32]; gene = "wt";  #done with M complete #EXAMPLE???
fPath = "20201113_5dpf_Rh2hG_s2mCh_uninj_i08d"; zlims=[28,30]; gene = "wt"; #done with M complete
fPath = "20201113_5dpf_Rh2hG_s2mCh_uninj_i09b"; zlims=[36,39]; gene = "wt"; #done with M complete
fPath = "20201113_5dpf_Rh2hG_s2mCh_uninj_i09d"; zlims=[32,34]; gene = "wt"; #done with M complete
fPath = "20201113_5dpf_Rh2hG_s2mCh_uninj_i10d"; zlims=[28,31]; gene = "wt"; #done S only
fPath = "20201113_5dpf_Rh2hG_s2mCh_uninjected_i11b"; zlims=[27,29]; gene = "wt"; #done S only # S cone channel is strange in this one
fPath = "20201113_5dpf_Rh2hG_s2mCh_uninjected_i14b"; zlims=[26,29]; gene = "wt"; #done S only

#### fPath = "20201113_5dpf_Rh2hG_s2mCh_uninj_i08b2"; zlims=[00,00]; gene = "wt";  # beautiful peripheral retina
#### fPath = "20201113_5dpf_Rh2hG_s2mCh_uninj_i10b"; zlims=[00,00]; gene = "wt"; #side-view
#### fPath = "20201113_5dpf_Rh2hG_s2mCh_uninj_i10e"; zlims=[00,00]; gene = "wt"; #side-view

# tbx2a


#tbx2b
#  is there a decrease in S cones?
fPath = "20201113_5dpf_Rh2hG_s2mCh_CRtbx2b_i04b"; zlims=[27,31]; gene = "tbx2b"; #done with M complete
fPath = "20201113_5dpf_Rh2hG_s2mCh_CRtbx2b_i04d"; zlims=[29,31]; gene = "tbx2b"; # done with S
fPath = "20201113_5dpf_Rh2hG_s2mCh_CRtbx2b_i05b"; zlims=[32,34]; gene = "tbx2b"; # done with S
fPath = "20201113_5dpf_Rh2hG_s2mCh_CRtbx2b_i05d"; zlims=[26,29]; gene = "tbx2b"; # done with S
fPath = "20201113_5dpf_Rh2hG_s2mCh_CRtbx2b_i06b"; zlims=[22,23]; gene = "tbx2b"; # done with S #extreme phenotype example
fPath = "20201113_5dpf_Rh2hG_s2mCh_CRtbx2b_i06d"; zlims=[42,44]; gene = "tbx2b"; # done with S
fPath = "20201113_5dpf_Rh2hG_s2mCh_CRtbx2b_i07b"; zlims=[25,27]; gene = "tbx2b"; # done with S #example for injected but no phenotype?
fPath = "20201113_5dpf_Rh2hG_s2mCh_CRtbx2b_i07d"; zlims=[28,30]; gene = "tbx2b"; # done with S
fPath = "20201113_5dpf_Rh2hG_s2mCh_CRtbx2b_i08b"; zlims=[40,42]; gene = "tbx2b"; # done with S
fPath = "20201113_5dpf_Rh2hG_s2mCh_CRtbx2b_i08d"; zlims=[29,31]; gene = "tbx2b"; # done with S


***
## Quantification: GFP inside S cones<a id='quantification'>∮</a>
***
[Back to Index](#Index)

Plan (loosely based on Mander's coefficient):
1. Erode S cone segmentation to focus on center of cells
2. Divide GFP fluorescence into millies as a way to normalize data between images into 0 - 1000 range
3. Calculate for each cone what normalized average fluorescence corresponds too.

Questions:
1. How much erosion?
2. Do I have to exclude S cones to do GFP percentile calculation? &rarr; this would assume that GFP outside L cones is unchanged, which could be tested|plotted directly.
3. Also check if RFP distributions are unchanged?

In [ ]:
def calcMillie(viewer,lname):
    outHist,outBins = np.histogram(viewer.layers[lname].data.flatten(), bins = 1000)
    return outHist,outBins

def GFPanalyzer(viewer,dAnalysis,gene,fPath, erosionValue=10):
    # clear viewer
    viewer.layers.select_all()
    viewer.layers.remove_selected()
    dOut = dAnalysis + gene + '/' + fPath + '/'
    print('Quantifying GFP in S cones of ' + fPath + '(' + gene + ')\n')
    # load mips
    viewer.open(dOut + 'M' + "_mip.tiff", plugin='builtins', colormap = 'green', blending='additive');
    viewer.open(dOut + 'S' + "_mip.tiff", plugin='builtins', colormap = 'magenta', blending='additive');
    # viewer.open(dOut + 'M' + "_seg_curated_incomplete.tiff", name='M_seg', plugin='builtins', blending='additive');
    viewer.open(dOut + 'S' + "_seg_curated.tiff", name='S_seg', plugin='builtins', blending='additive');
    # viewer.layers['M_seg'].preserve_labels = True;
    viewer.layers['S_seg'].preserve_labels = True;
    viewer.layers['S_seg'].contour = 5
    # erode L cone segmentation
    viewer.layers['S_seg'].data = ndimage.grey_erosion(viewer.layers['S_seg'].data, size=(erosionValue,erosionValue))
    # divide images into 1000 bins
    mH, mB = calcMillie(viewer,'M_mip')
    sH, sB = calcMillie(viewer,'S_mip')

    scones = np.unique(viewer.layers['S_seg'].data)
    scones = scones[scones>0]
    sRFP = np.empty(np.shape(scones))
    sGFP = np.empty(np.shape(scones))
    sRFPsd = np.empty(np.shape(scones))
    sGFPsd = np.empty(np.shape(scones))
    sRtile = np.empty(np.shape(scones))
    sGtile = np.empty(np.shape(scones))
    
    i=0;
    for s in scones:
        sRFP[i] = np.mean(viewer.layers['S_mip'].data[viewer.layers['S_seg'].data==s])
        sGFP[i] = np.mean(viewer.layers['M_mip'].data[viewer.layers['S_seg'].data==s])
        sRFPsd[i] = np.std(viewer.layers['S_mip'].data[viewer.layers['S_seg'].data==s])
        sGFPsd[i] = np.std(viewer.layers['M_mip'].data[viewer.layers['S_seg'].data==s])
        sRtile[i] = len(sB[sB<sRFP[i]])
        sGtile[i] = len(mB[mB<sGFP[i]])
        i=i+1;
        
    np.savez(dOut + 'quantificationGFP.npz',
             scones=scones,
             sRFP=sRFP,
             sGFP=sGFP,
             sRFPsd=sRFPsd,
             sGFPsd=sGFPsd,
             sRtile=sRtile,
             sGtile=sGtile)
    results = zip(['scones','sRFP','sGFP','sRFPsd','sGFPsd','sRtile','sGtile'],
                 [scones,sRFP,sGFP,sRFPsd,sGFPsd,sRtile,sGtile])
    
    return results

In [ ]:
erodeV = 12;

# analysis directory
dAnalysis = "/Users/angueyraaristjm/Documents/LiImaging/Analysis/CRtbx2F0s/m2Gs2C/"

# open file

# uninjected
### 20201009: imaged 3dpf and 5dpf larvae. Analyzing 5 dpf for paper
gene = 'wt'
wt_fPaths = ["20201113_5dpf_Rh2hG_s2mCh_uninj_i07b",
             "20201113_5dpf_Rh2hG_s2mCh_uninj_i07d",
             "20201113_5dpf_Rh2hG_s2mCh_uninj_i08b",
             "20201113_5dpf_Rh2hG_s2mCh_uninj_i08d",
             "20201113_5dpf_Rh2hG_s2mCh_uninj_i09b",
             "20201113_5dpf_Rh2hG_s2mCh_uninj_i09d",
             "20201113_5dpf_Rh2hG_s2mCh_uninj_i10d",
             "20201113_5dpf_Rh2hG_s2mCh_uninjected_i11b",
             "20201113_5dpf_Rh2hG_s2mCh_uninjected_i14b",
             "20210108_5dpf_Rh2hG_s2mCh_uninjected_i01b",
             "20210108_5dpf_Rh2hG_s2mCh_uninjected_i02b",
             "20210108_5dpf_Rh2hG_s2mCh_uninjected_i03b",
             "20210108_5dpf_Rh2hG_s2mCh_uninjected_i04b",
             "20210108_5dpf_Rh2hG_s2mCh_zpr3_647_uninjected_i06d",
             "20210108_5dpf_Rh2hG_s2mCh_zpr3_647_uninjected_i07b",
         ]
for fPath in wt_fPaths:
    GFPanalyzer(viewer,dAnalysis,gene,fPath, erosionValue=erodeV)
    
# tbx2b
gene = 'tbx2b';
tbx2b_fPaths = ["20201113_5dpf_Rh2hG_s2mCh_CRtbx2b_i04b",
                "20201113_5dpf_Rh2hG_s2mCh_CRtbx2b_i04d",
                "20201113_5dpf_Rh2hG_s2mCh_CRtbx2b_i05b",
                "20201113_5dpf_Rh2hG_s2mCh_CRtbx2b_i05d",
                "20201113_5dpf_Rh2hG_s2mCh_CRtbx2b_i06b",
                "20201113_5dpf_Rh2hG_s2mCh_CRtbx2b_i06d",
                "20201113_5dpf_Rh2hG_s2mCh_CRtbx2b_i07b",
                "20201113_5dpf_Rh2hG_s2mCh_CRtbx2b_i07d",
                "20201113_5dpf_Rh2hG_s2mCh_CRtbx2b_i08b",
                "20201113_5dpf_Rh2hG_s2mCh_CRtbx2b_i08d",
               ]
for fPath in tbx2b_fPaths:
    GFPanalyzer(viewer,dAnalysis,gene,fPath, erosionValue=erodeV)

    
# tbx2a
gene = 'tbx2a';
tbx2a_fPaths = ["20210108_5dpf_Rh2hG_s2mCh_CRtbx2a_i01b",
                "20210108_5dpf_Rh2hG_s2mCh_CRtbx2a_i01d",
                "20210108_5dpf_Rh2hG_s2mCh_CRtbx2a_i02b",
                "20210108_5dpf_Rh2hG_s2mCh_CRtbx2a_i02d",
                "20210108_5dpf_Rh2hG_s2mCh_mRho647_CRtbx2a_i20b",
                "20210108_5dpf_Rh2hG_s2mCh_mRho647_CRtbx2a_i21b",
                "20210108_5dpf_Rh2hG_s2mCh_mRho647_CRtbx2a_i21d",
                "20210108_5dpf_Rh2hG_s2mCh_mRho647_CRtbx2a_i25b",
                "20210108_5dpf_Rh2hG_s2mCh_zpr3_647_CRtbx2a_i13b",
               ]
for fPath in tbx2a_fPaths:
    GFPanalyzer(viewer,dAnalysis,gene,fPath, erosionValue=erodeV)

    

print('Finished analysis')

In [ ]:

# collect all results
# analysis directory
dAnalysis = "/Users/angueyraaristjm/Documents/LiImaging/Analysis/CRtbx2F0s/m2Gs2C/"
wt_fPaths = ["20201113_5dpf_Rh2hG_s2mCh_uninj_i07b",
             "20201113_5dpf_Rh2hG_s2mCh_uninj_i07d",
             "20201113_5dpf_Rh2hG_s2mCh_uninj_i08b",
             "20201113_5dpf_Rh2hG_s2mCh_uninj_i08d",
             "20201113_5dpf_Rh2hG_s2mCh_uninj_i09b",
             "20201113_5dpf_Rh2hG_s2mCh_uninj_i09d",
             "20201113_5dpf_Rh2hG_s2mCh_uninj_i10d",
             "20201113_5dpf_Rh2hG_s2mCh_uninjected_i11b",
             "20201113_5dpf_Rh2hG_s2mCh_uninjected_i14b",
             "20210108_5dpf_Rh2hG_s2mCh_uninjected_i01b",
             "20210108_5dpf_Rh2hG_s2mCh_uninjected_i02b",
             "20210108_5dpf_Rh2hG_s2mCh_uninjected_i03b",
             "20210108_5dpf_Rh2hG_s2mCh_uninjected_i04b",
             "20210108_5dpf_Rh2hG_s2mCh_zpr3_647_uninjected_i06d",
             "20210108_5dpf_Rh2hG_s2mCh_zpr3_647_uninjected_i07b",
         ]
    
# tbx2b
gene = 'tbx2b';
tbx2b_fPaths = ["20201113_5dpf_Rh2hG_s2mCh_CRtbx2b_i04b",
                "20201113_5dpf_Rh2hG_s2mCh_CRtbx2b_i04d",
                "20201113_5dpf_Rh2hG_s2mCh_CRtbx2b_i05b",
                "20201113_5dpf_Rh2hG_s2mCh_CRtbx2b_i05d",
                "20201113_5dpf_Rh2hG_s2mCh_CRtbx2b_i06b",
                "20201113_5dpf_Rh2hG_s2mCh_CRtbx2b_i06d",
                "20201113_5dpf_Rh2hG_s2mCh_CRtbx2b_i07b",
                "20201113_5dpf_Rh2hG_s2mCh_CRtbx2b_i07d",
                "20201113_5dpf_Rh2hG_s2mCh_CRtbx2b_i08b",
                "20201113_5dpf_Rh2hG_s2mCh_CRtbx2b_i08d",
               ]

   
# tbx2a
gene = 'tbx2a';
tbx2a_fPaths = ["20210108_5dpf_Rh2hG_s2mCh_CRtbx2a_i01b",
                "20210108_5dpf_Rh2hG_s2mCh_CRtbx2a_i01d",
                "20210108_5dpf_Rh2hG_s2mCh_CRtbx2a_i02b",
                "20210108_5dpf_Rh2hG_s2mCh_CRtbx2a_i02d",
                "20210108_5dpf_Rh2hG_s2mCh_mRho647_CRtbx2a_i20b",
                "20210108_5dpf_Rh2hG_s2mCh_mRho647_CRtbx2a_i21b",
                "20210108_5dpf_Rh2hG_s2mCh_mRho647_CRtbx2a_i21d",
                "20210108_5dpf_Rh2hG_s2mCh_mRho647_CRtbx2a_i25b",
                "20210108_5dpf_Rh2hG_s2mCh_zpr3_647_CRtbx2a_i13b",
               ]


rName = 'sGtile'

wtResults = []
i=1;
for fPath in wt_fPaths:
    gene = 'wt'
    dOut = dAnalysis + gene + '/' + fPath + '/'
    results = np.load(dOut + 'quantificationGFP.npz')
    wtResults.append(results[rName])
    i=i+1;

bResults = []
i=1;
for fPath in tbx2b_fPaths:
    gene = 'tbx2b'
    dOut = dAnalysis + gene + '/' + fPath + '/'
    results = np.load(dOut + 'quantificationGFP.npz')
    bResults.append(results[rName])
    i=i+1;


aResults = []
i=1;
for fPath in tbx2a_fPaths:
    gene = 'tbx2a'
    dOut = dAnalysis + gene + '/' + fPath + '/'
    results = np.load(dOut + 'quantificationGFP.npz')
    aResults.append(results[rName])
    i=i+1;


# combine across images
wtResults = np.concatenate(wtResults).ravel()
aResults = np.concatenate(aResults).ravel()
bResults = np.concatenate(bResults).ravel()

# plot histogram
pdfV = True
# wtH, wtB = np.histogram(wtResults, bins = 50, density=pdfV)
# aH, aB = np.histogram(aResults, bins = 50, density=pdfV)
# bH, bB = np.histogram(bResults, bins = 50, density=pdfV)

# plot histogram but first calculate a fixed bin number for all genotypes
# allBins = np.histogram_bin_edges(np.concatenate([wtResults,aResults,bResults]).ravel(),bins='fd');
allBins = np.arange(0,1010,10);
# allBins = np.histogram_bin_edges(np.concatenate([wtResults,bResults]).ravel(),bins='fd');
allBinsC = (allBins[:-1] + allBins[1:]) / 2 # Centers
wtH, wtB = np.histogram(wtResults, bins = allBins, density=pdfV)
aH, aB = np.histogram(aResults, bins = allBins, density=pdfV)
bH, bB = np.histogram(bResults, bins = allBins, density=pdfV)

# # this can be used for cumulative distribution then normalized
# wtH = np.cumsum(wtH)/np.max(np.cumsum(wtH));
# aH = np.cumsum(aH)/np.max(np.cumsum(aH));
# bH = np.cumsum(bH)/np.max(np.cumsum(bH));

wt95CI = np.min(allBinsC[(np.cumsum(wtH))>.095])/1000;
wGBright = np.sum(wtH[allBinsC>wt95CI*1000]);
aGBright = np.sum(aH[allBinsC>wt95CI*1000]);
bGBright = np.sum(bH[allBinsC>wt95CI*1000]);

baseColor = 'k'
if (plotStyle=='light'):
    baseColor = 'k';

fH, (axH1,axH2,axH3) = plt.subplots(nrows=1, ncols=3, figsize=(30, 8))
axH1.stairs(wtH,wtB/1000, fill=True, color = baseColor, alpha = 1.0)
axH2.stairs(wtH,wtB/1000, fill=True, color = baseColor, alpha = 0.5)
axH3.stairs(wtH,wtB/1000, fill=True, color = baseColor, alpha = 0.5)
axH2.stairs(aH,aB/1000, fill=True, color = zfC['lws1'], alpha = 1)
axH3.stairs(bH,bB/1000, fill=True, color = zfC['sws2'], alpha = 1)

axH1.plot([wt95CI,wt95CI],[0,0.008],color=baseColor)
axH2.plot([wt95CI,wt95CI],[0,0.008],color=baseColor)
axH3.plot([wt95CI,wt95CI],[0,0.008],color=baseColor)

formatFigureMain(fH, axH1, _)
formatFigureMain(fH, axH2, _)
formatFigureMain(fH, axH3, _)
# axH.set_title('Distribution of GFP inside L cones')
for axH in (axH1,axH2,axH3):
    axH.set_xlabel('Normalized GFP values')
    if pdfV:
        axH.set_ylabel('Probability density')
    else:
        axH.set_ylabel('Counts')
## normalized
# axH.stairs(wtH/np.max(wtH),wtB, fill=True, color = 'k', alpha = 0.5)
# axH.stairs(aH/np.max(aH),aB, fill=True, color = 'r', alpha = 0.5)
# axH.stairs(bH/np.max(bH),bB, fill=True, color = 'b', alpha = 0.5)

# dPath = '/Users/angueyraaristjm/Documents/LiLab/Manuscripts/2021_zfConeRNAseq/resources/'
dPath = '/Users/angueyraaristjm/Documents/LiLab/Presentations/revealjs/resources/20211008_UCLA/'
plt.savefig(dPath + "CRtbx2F0_SM_hist.svg", transparent=True, format="svg")

print('# cells of bright GFP cells: wt={0}, tbx2a={1}, tbx2b={2}'.format(wGBright*1000,aGBright*1000,bGBright*1000))


In [ ]:
res = mannwhitneyu(wtH, aH, alternative="less", method="auto")
print(res)

In [ ]:
res = mannwhitneyu(wtH, bH, alternative="less", method="auto")
print(res)

## Testing Laura's buggy code

In [ ]:
def calcMillie(viewer,lname):
    outHist,outBins = np.histogram(viewer.layers[lname].data.flatten(), bins = 1000)
    return outHist,outBins

 

def GFPanalyzer(viewer,dAnalysis,gene,fPath, erosionValue=10):
    # clear viewer
    viewer.layers.select_all()
    viewer.layers.remove_selected()
    dOut = dAnalysis + gene + '/' + fPath + '/'
    print('Quantifying GFP in S cones of ' + fPath + '(' + gene + ')\n')
    # load mips
    viewer.open(dOut + 'M' + "_mip.tiff", plugin='builtins', colormap = 'green', blending='additive');
    viewer.open(dOut + 'S' + "_mip.tiff", plugin='builtins', colormap = 'magenta', blending='additive');
    # viewer.open(dOut + 'M' + "_seg_curated_incomplete.tiff", name='M_seg', plugin='builtins', blending='additive');
    viewer.open(dOut + 'S' + "_seg_curated.tiff", name='S_seg', plugin='builtins', blending='additive');
    # viewer.layers['M_seg'].preserve_labels = True;
    viewer.layers['S_seg'].preserve_labels = True;
    viewer.layers['S_seg'].contour = 5
    # erode L cone segmentation
    viewer.layers['S_seg'].data = ndimage.grey_erosion(viewer.layers['S_seg'].data, size=(erosionValue,erosionValue))
    # divide images into 1000 bins
    mH, mB = calcMillie(viewer,'M_mip')
    sH, sB = calcMillie(viewer,'S_mip')

 

    scones = np.unique(viewer.layers['S_seg'].data)
    scones = scones[scones>0]
    sRFP = np.empty(np.shape(scones))
    sGFP = np.empty(np.shape(scones))
    sRFPsd = np.empty(np.shape(scones))
    sGFPsd = np.empty(np.shape(scones))
    sRtile = np.empty(np.shape(scones))
    sGtile = np.empty(np.shape(scones))
    
    i=0;
    for s in scones:
        sRFP[i] = np.mean(viewer.layers['S_mip'].data[viewer.layers['S_seg'].data==s])
        sGFP[i] = np.mean(viewer.layers['M_mip'].data[viewer.layers['S_seg'].data==s])
        sRFPsd[i] = np.std(viewer.layers['S_mip'].data[viewer.layers['S_seg'].data==s])
        sGFPsd[i] = np.std(viewer.layers['M_mip'].data[viewer.layers['S_seg'].data==s])
        sRtile[i] = len(sB[sB<sRFP[i]])
        sGtile[i] = len(mB[mB<sGFP[i]])
        i=i+1;
    
    np.savez(dOut + 'quantificationGFP.npz',
             scones=scones,
             sRFP=sRFP,
             sGFP=sGFP,
             sRFPsd=sRFPsd,
             sGFPsd=sGFPsd,
             sRtile=sRtile,
             sGtile=sGtile)
    results = zip(['scones','sRFP','sGFP','sRFPsd','sGFPsd','sRtile','sGtile'],
                 [scones,sRFP,sGFP,sRFPsd,sGFPsd,sRtile,sGtile])

 

    return results

In [ ]:
erodeV = 12;

 

# analysis directory
dAnalysis = "/Users/angueyraaristjm/Documents/LiImaging/Analysis/CRfoxq2F0/mw2Gs2C/"
# dAnalysis = "C:/Users/pataklk/Documents/F0_Analysis/CRfoxq2F0/mw2Gs2C/"
# open file

 

# uninjected
### 20201009: imaged 3dpf and 5dpf larvae. Analyzing 5 dpf for paper
# gene = 'wt'
# wt_fPaths = ["20210430_5dpf_m2GFP_s2mCh_uninj_i01b001",
#                 "20210430_5dpf_m2GFP_s2mCh_uninj_i01d",
#                 "20210430_5dpf_m2GFP_s2mCh_uninj_i02b",
#                 "20210430_5dpf_m2GFP_s2mCh_uninj_i02d",
#                 "20210430_5dpf_m2GFP_s2mCh_uninj_i03b",
#                 "20210430_5dpf_m2GFP_s2mCh_uninj_i03d",
#                 "20210430_5dpf_m2GFP_s2mCh_uninj_i04b",
#                 "20210430_5dpf_m2GFP_s2mCh_uninj_i05b",
#                 "20210430_5dpf_m2GFP_s2mCh_uninj_i05d",
#                ]
# for fPath in wt_fPaths:
#     GFPanalyzer(viewer,dAnalysis,gene,fPath, erosionValue=erodeV)
    
    
# # foxq2
gene = 'foxq2';
foxq2_fPaths = ["20210430_5dpf_m2GFP_s2mCh_CRfoxq2_i01b",
                "20210430_5dpf_m2GFP_s2mCh_CRfoxq2_i01d",
                "20210430_5dpf_m2GFP_s2mCh_CRfoxq2_i02b",
                "20210430_5dpf_m2GFP_s2mCh_CRfoxq2_i02d",
                #"20210430_5dpf_m2GFP_s2mCh_CRfoxq2_i03b",
                "20210430_5dpf_m2GFP_s2mCh_CRfoxq2_i03d",
                "20210430_5dpf_m2GFP_s2mCh_CRfoxq2_i04b",
                "20210430_5dpf_m2GFP_s2mCh_CRfoxq2_i04d",
                "20210430_5dpf_m2GFP_s2mCh_CRfoxq2_i06d",
                "20210430_5dpf_m2GFP_s2mCh_CRfoxq2_i06d",
                "20210430_5dpf_m2GFP_s2mCh_CRfoxq2_i07b",
                "20210430_5dpf_m2GFP_s2mCh_CRfoxq2_i07d",
                #"20210430_5dpf_m2GFP_s2mCh_CRfoxq2_i08b",
                "20210430_5dpf_m2GFP_s2mCh_CRfoxq2_i08d",
               ]

 

for fPath in foxq2_fPaths:
    tempresults = GFPanalyzer(viewer,dAnalysis,gene,fPath, erosionValue=erodeV)

 


print('Finished analysis')

In [ ]:
tempresults['scones']

In [ ]:
# collect all results
# analysis directory
dAnalysis = "C:/Users/pataklk/Documents/F0_Analysis/CRfoxq2F0/mw2Gs2C/"
dAnalysis = "/Users/angueyraaristjm/Documents/LiImaging/Analysis/CRfoxq2F0/mw2Gs2C/"
wt_fPaths = ["20210430_5dpf_m2GFP_s2mCh_uninj_i01b001",
                "20210430_5dpf_m2GFP_s2mCh_uninj_i01d",
                "20210430_5dpf_m2GFP_s2mCh_uninj_i02b",
                "20210430_5dpf_m2GFP_s2mCh_uninj_i02d",
                "20210430_5dpf_m2GFP_s2mCh_uninj_i03b",
                "20210430_5dpf_m2GFP_s2mCh_uninj_i03d",
                "20210430_5dpf_m2GFP_s2mCh_uninj_i04b",
                "20210430_5dpf_m2GFP_s2mCh_uninj_i05b",
                "20210430_5dpf_m2GFP_s2mCh_uninj_i05d",
            ]
    
# tbx2b
gene = 'foxq2';
foxq2_fPaths = ["20210430_5dpf_m2GFP_s2mCh_CRfoxq2_i01b",
                "20210430_5dpf_m2GFP_s2mCh_CRfoxq2_i01d",
                "20210430_5dpf_m2GFP_s2mCh_CRfoxq2_i02b",
                "20210430_5dpf_m2GFP_s2mCh_CRfoxq2_i02d",
                #"20210430_5dpf_m2GFP_s2mCh_CRfoxq2_i03b",
                "20210430_5dpf_m2GFP_s2mCh_CRfoxq2_i03d",
                "20210430_5dpf_m2GFP_s2mCh_CRfoxq2_i04b",
                "20210430_5dpf_m2GFP_s2mCh_CRfoxq2_i04d",
#                 "20210430_5dpf_m2GFP_s2mCh_CRfoxq2_i06d",
                "20210430_5dpf_m2GFP_s2mCh_CRfoxq2_i06d",
                "20210430_5dpf_m2GFP_s2mCh_CRfoxq2_i07b",
                "20210430_5dpf_m2GFP_s2mCh_CRfoxq2_i07d",
                #"20210430_5dpf_m2GFP_s2mCh_CRfoxq2_i08b",
                "20210430_5dpf_m2GFP_s2mCh_CRfoxq2_i08d",
               ]
#C:\Users\pataklk\Documents\F0_Analysis\CRfoxq2F0\mw2Gs2C\foxq2\20210430_5dpf_m2GFP_s2mCh_CRfoxq2_i01b

 

rName = 'sGtile'

 

wtResults = []
i=1;
for fPath in wt_fPaths:
    gene = 'wt'
    dOut = dAnalysis + gene + '/' + fPath + '/'
    results = np.load(dOut + 'quantificationGFP.npz')
    wtResults.append(results[rName])
    i=i+1

 
fResults = []
i=1;
for fPath in foxq2_fPaths:
    gene = 'foxq2'
    dOut = dAnalysis + gene + '/' + fPath + '/'
    results = np.load(dOut + 'quantificationGFP.npz')
    fResults.append(results[rName])
    i=i+1

 

# combine across images
wtResults = np.concatenate(wtResults).ravel()
fResults = np.concatenate(fResults).ravel()
#bResults = np.concatenate(bResults).ravel()
print(fResults)

In [ ]:
# plot histogram
pdfV = False
# wtH, wtB = np.histogram(wtResults, bins = 50, density=pdfV)
# fH, fB = np.histogram(fResults, bins = 50, density=pdfV)


# plot histogram but first calculate a fixed bin number for all genotypes
# allBins = np.histogram_bin_edges(np.concatenate([wtResults,aResults,bResults]).ravel(),bins='fd');
allBins = np.arange(0,1010,10);
# allBins = np.histogram_bin_edges(np.concatenate([wtResults,fResults]).ravel(),bins='fd');
allBinsC = (allBins[:-1] + allBins[1:]) / 2 # Centers
wtH, wtB = np.histogram(wtResults, bins = allBins, density=pdfV)
fqH, fqB = np.histogram(fResults, bins = allBins, density=pdfV)


# # this can be used for cumulative distribution then normalized
# wtH = np.cumsum(wtH)/np.max(np.cumsum(wtH));
# aH = np.cumsum(aH)/np.max(np.cumsum(aH));
# bH = np.cumsum(bH)/np.max(np.cumsum(bH));

wt95CI = np.min(allBinsC[(np.cumsum(wtH))>.095])/1000;
wGBright = np.sum(wtH[allBinsC>wt95CI*1000]);
fGBright = np.sum(fqH[allBinsC>wt95CI*1000]);

baseColor = 'w'
if (plotStyle=='light'):
    baseColor = 'k';

fH, (axH1,axH2) = plt.subplots(nrows=1, ncols=2, figsize=(30, 8))
axH1.stairs(wtH,wtB/1000, fill=True, color = baseColor, alpha = 1.0)
axH2.stairs(wtH,wtB/1000, fill=True, color = baseColor, alpha = 0.8)
axH2.stairs(fqH,fqB/1000, fill=True, color = zfC['sws2'], alpha = 0.75)

axH1.plot([wt95CI,wt95CI],[0,0.014],color=baseColor)
axH2.plot([wt95CI,wt95CI],[0,0.014],color=baseColor)


formatFigureMain(fH, axH1, _)
formatFigureMain(fH, axH2, _)
# formatFigureMain(fH, axH3, _)
# # axH.set_title('Distribution of GFP inside L cones')
for axH in (axH1,axH2,axH3):
    axH.set_xlabel('Normalized GFP values')
    if pdfV:
        axH.set_ylabel('Probability density')
    else:
        axH.set_ylabel('Counts')
## normalized
# axH.stairs(wtH/np.max(wtH),wtB, fill=True, color = 'k', alpha = 0.5)
# axH.stairs(aH/np.max(aH),aB, fill=True, color = 'r', alpha = 0.5)
# axH.stairs(bH/np.max(bH),bB, fill=True, color = 'b', alpha = 0.5)

dPath = '/Users/angueyraaristjm/Documents/LiLab/Manuscripts/2021_zfConeRNAseq/resources/'
# dPath = '/Users/angueyraaristjm/Documents/LiLab/Conferences/2021_RCS/resources/'
# plt.savefig(dPath + "CRtbx2F0_SM_hist.svg", transparent=True, format="svg")

print('# cells of bright GFP cells: wt={0}, foxq2={1}'.format(wGBright*1000,fGBright*1000))

In [ ]:
# divide images into 1000 bins
mH, mB = np.histogram(viewer.layers['M_mip'].data.flatten(), bins = 1000)
sH, sB = np.histogram(viewer.layers['S_mip'].data.flatten(), bins = 1000)


fH, axH = plt.subplots(figsize=(8,8))
axH.stairs(sH,sB, fill=True, color = 'm', alpha = 0.5)
axH.stairs(mH,mB, fill=True, color = 'g', alpha = 0.5)

In [ ]:
viewer.close()

In [ ]:
viewer.close()

## Testing segmentation
1. Load single plane image from zoon in into larval eye, L cones only
2. Run cellpose
3. Count number of 

In [ ]:
viewer = napari.Viewer()

In [ ]:
# clear viewer
viewer.layers.select_all()
viewer.layers.remove_selected()

In [ ]:
dPath = "/Users/angueyraaristjm/Documents/LiImaging/A1R/zf_partial/20201009_CRtbx2a/"
fPathL = "20201009_i06b_L"
fPathM = "20201009_i06b_M"
# load the image data and inspect its shape
zfL = imread(dPath+fPathL + ".tiff")
zfM = imread(dPath+fPathM + ".tiff")
viewer.add_image(zfL, colormap='magenta')#, contrast_limits=[196,5860]);
viewer.add_image(zfM, colormap='green', blending='additive')#, contrast_limits=[223,6721]);
viewer.open(dPath+fPathL + "_seg.tiff", name='zfL_seg', plugin='builtins');
viewer.open(dPath+fPathM + "_seg.tiff", name='zfM_seg', plugin='builtins');
# nbscreenshot(viewer)

In [ ]:
nL = len(np.unique(viewer.layers['zfL_seg'].data))-1
print("Image has {0} L cones".format(nL))
nM = len(np.unique(viewer.layers['zfM_seg'].data))-1
print("Image has {0} M cones".format(nM))

dL = viewer.layers['zfL_seg'].data
dM = viewer.layers['zfM_seg'].data

Not sure how to normalize across images...
1. average GFP instensity inside M-cones? &rarr; difficult in CRISPants
1. max GFP intensity? &rarr; could be ok

In [ ]:
# eroding segmentation because it too generous with ROIs and I want to exclude GFP diffraction
viewer.layers['zfL_seg'].data = ndimage.grey_erosion(dL, size=(8,8))
viewer.layers['zfM_seg'].data = ndimage.grey_erosion(dM, size=(8,8))

L_avgG = np.zeros([nL,1])
L_avgR = np.zeros([nL,1])
# calculate mean GFP intensity inside each L cone (normalized and baseline subtracted)
maxM = viewer.layers['zfM'].data[viewer.layers['zfL_seg'].data == 0].max()
maxL = viewer.layers['zfL'].data[viewer.layers['zfL_seg'].data == 1].max()
minM = viewer.layers['zfM'].data[viewer.layers['zfL_seg'].data == 0].min()
minL = viewer.layers['zfL'].data[viewer.layers['zfL_seg'].data == 0].min()
for roi in range(nL):
    tempROI = viewer.layers['zfL_seg'].data == roi
    L_avgG[roi] = (viewer.layers['zfM'].data[tempROI].mean()-minM)/(maxM-minM)
    L_avgR[roi] = (viewer.layers['zfL'].data[tempROI].mean()-minL)/(maxL-minL)

M_avgG = np.zeros([nM,1])
M_avgR = np.zeros([nM,1])
# calculate mean GFP intensity inside each M cone (normalized and baseline subtracted)
for roi in range(nM):
    tempROI = viewer.layers['zfM_seg'].data == roi
    M_avgG[roi] = (viewer.layers['zfM'].data[tempROI].mean()-minM)/(maxM-minM)
    M_avgR[roi] = (viewer.layers['zfL'].data[tempROI].mean()-minL)/(maxL-minL)

    
nbins = 20
denseFlag = True
    
plt.figure(figsize=(8,6))
plt.hist(L_avgG, bins=np.divide(range(0,nbins+1),nbins), density = denseFlag, color = (1.0,0.5,0.0,0.5), label="L_GFP")
plt.hist(L_avgR, bins=np.divide(range(0,nbins+1),nbins), density = denseFlag, color = (1.0,0.0,0.0,0.5), label="L_tdT")

plt.hist(M_avgG, bins=np.divide(range(0,nbins+1),nbins), density = denseFlag, color = (0.0,1.0,0.0,0.5), label="M_GFP")
plt.hist(M_avgR, bins=np.divide(range(0,nbins+1),nbins), density = denseFlag, color = (0.5,1.0,0.0,0.5), label="M_tdT")

plt.xlabel("Normalized intensity", size=14)
plt.ylabel("Probability", size=14)
plt.title("")
plt.legend(loc='upper right')
# plt.savefig("overlapping_histograms_with_matplotlib_Python.png")

In [ ]:
# viewer.layers['zfM_seg'].data = dM
viewer.layers['zfL_seg'].data = ndimage.grey_erosion(viewer.layers['zfL_seg'].data, size=(10,10))

In [ ]:
@viewer.bind_key('k')
def toggle_sel(viewer):
    if (viewer.layers['zfM_seg'].show_selected_label == True):
        viewer.layers['zfM_seg'].show_selected_label = False
    elif (viewer.layers['zfM_seg'].show_selected_label == False):
        viewer.layers['zfM_seg'].show_selected_label = True

In [ ]:
viewer.layers['zfM_seg'].show_selected_label=False

In [ ]:
viewer.layers['zfM_seg'].data